# Travel Time Analytics: TIME SERIES DECOMPOSITION
### This notebook applies time series decomosition to travel times from XD segments at traffic signals

First, all files in a folder containing the travel time data are imported and combined into a dataframe.
The file format used is parquet, because it is astonishingly faster than csv for both reading and writing, and is also compatible with Power BI, and it saves datatypes

The the reference speed for each segment, which is genererated by INRIX, is extracted and saved to a dimension file dim_signals_XD, and that will be used to calculate travel time index in Power BI.

Then each XD segment is decomposed into trend, seasonal, and residual components.

Finally, the resulting dataframe is saved to a single parquet file, ready to be imported into Power BI for visualization.



In [1]:
import pandas as pd
import glob
import statsmodels.api as sm

#from matplotlib import pyplot as plt
#import datetime as dt
#import datatable
#from multiprocessing import Process
#import pyarrow


Alternative method to below for reading all files to a dataframe, may update later:
df = pd.concat(map(pd.read_csv, glob.glob('//scdata2/signalshar/Data_Analysis/INRIX_API/Speed_Data/Daily_Download/*.csv')))

In [2]:
#read all csv files into dataframe. Awesome! 
folder = r'//scdata2/signalshar/Data_Analysis/INRIX_API/Speed_Data/Daily_Download/'
all_files = glob.glob(folder + "*.parquet")
li = []

for filename in all_files:
    df = pd.read_parquet(filename) #, parse_dates=['Date Time'], header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
del li
frame

,Date Time,Segment ID,Speed(miles/hour),Ref Speed(miles/hour),Travel Time(Minutes),CValue
0,2021-11-01 00:00:00,120645869,15,15,0.27,NaN
1,2021-11-01 00:15:00,120645869,15,15,0.27,NaN
2,2021-11-01 00:30:00,120645869,15,15,0.27,NaN
3,2021-11-01 00:45:00,120645869,15,15,0.27,NaN
4,2021-11-01 01:00:00,120645869,15,15,0.27,NaN
...,...,...,...,...,...,...
17123395,2021-12-14 22:45:00,1237021323,31,31,0.08,100.0
17123396,2021-12-14 23:00:00,1237021323,31,31,0.08,100.0
17123397,2021-12-14 23:15:00,1237021323,35,31,0.07,100.0
17123398,2021-12-14 23:30:00,1237021323,36,31,0.07,100.0


In [3]:
# Extract reference speed for each segment, and create list of uniqe segments for loop
speed = frame.rename(columns={'Segment ID' : 'XDSegID'}).drop(columns = ['CValue', 'Speed(miles/hour)', 'Travel Time(Minutes)'])
segments = set(speed.XDSegID)
speed = speed.groupby('XDSegID').mean()
print(len(segments))
print(len(speed))

4050
4050


In [6]:
#Load dim_signals_XD, the dimension table which maps XD segments to signals, and join to it the reference speeds from above.
#This step only works if the latest speed data has already been downloaded
dim_signals_XD=pd.read_csv('//scdata2/signalshar/Data_Analysis/INRIX_API/Speed_Data/dim_signals_XD.csv', index_col='XDSegID')
print(len(dim_signals_XD.index))
dim_signals_XD = dim_signals_XD.merge(speed, on='XDSegID', how='left')
dim_signals_XD['reference_travel_time'] = dim_signals_XD['Miles'].div(dim_signals_XD['Ref Speed(miles/hour)']).mul(60)
print(len(dim_signals_XD.index))
#Save the new table to folder where a Power BI data gateway has been installed
dim_signals_XD.to_csv('//scdata2/signalshar/Data_Analysis/Data/Performance/dim_signals_XD.csv')#, index=False)
dim_signals_XD


4132
4132


,OID,PreviousXD,NextXDSegI,FRC,ToExport,Miles,Lanes,RoadNumber,RoadName,Country,...,Latitude,Longitude,key,_merge,AADT,SPEED,travel_time_minutes,group,Ref Speed(miles/hour),reference_travel_time
XDSegID,,,,,,,,,,,,,,,,,,,,,
116337789,6408164,NaN,NaN,5,NaN,0.111931,1.296,NaN,NE GOODFELLOW ST,UNITED STATES,...,44.024750,-116.943451,14037-116337789,left_only,NaN,NaN,NaN,1,11.0,0.610534
120916743,6520609,NaN,NaN,5,NaN,0.080331,1.068,NaN,SE GOODFELLOW ST,UNITED STATES,...,44.024750,-116.943451,14037-120916743,left_only,NaN,NaN,NaN,2,9.0,0.535540
383985787,8663597,1.187523e+09,4.289573e+08,2,NaN,0.422287,3.164,30.0,E IDAHO AVE,UNITED STATES,...,44.024750,-116.943451,14037-383985787,left_only,23600.000000,35.000000,0.723921,3,28.0,0.904901
1187436457,10961771,4.289573e+08,1.187436e+09,2,NaN,0.575795,3.237,30.0,E IDAHO AVE,UNITED STATES,...,44.024750,-116.943451,14037-1187436457,left_only,23600.000000,35.000000,0.987076,4,27.0,1.279544
116342301,6408295,NaN,NaN,5,NaN,0.062159,NaN,NaN,NE 4TH ST,UNITED STATES,...,44.025515,-116.956193,14034-116342301,left_only,NaN,NaN,NaN,5,5.0,0.745908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237060543,11220210,1.255757e+08,1.236967e+09,2,NaN,0.969812,3.770,26.0,US-26,UNITED STATES,...,45.400374,-122.278200,2C035-1237060543,left_only,32800.000000,36.666667,1.586965,2306,34.0,1.711432
1237060543,11220210,1.255757e+08,1.236967e+09,2,NaN,0.969812,3.770,26.0,US-26,UNITED STATES,...,45.402501,-122.284288,2C040-1237060543,left_only,32800.000000,36.666667,1.586965,2306,34.0,1.711432
1237060545,11220211,1.237061e+09,1.236983e+09,2,NaN,0.576977,3.296,26.0,US-26,UNITED STATES,...,45.402501,-122.284288,2C040-1237060545,left_only,32933.333333,43.333333,0.798891,899,36.0,0.961628


In [7]:
data = pd.DataFrame([])
#segments=[1237022307, 440963878]
for s in segments:
    df = frame[frame['Segment ID'] == s].set_index('Date Time').resample('H').mean().ffill()
    comp = sm.tsa.seasonal_decompose(x=df['Travel Time(Minutes)'], period=24*7, two_sided=False) #use .values to override pandas frequency, bc T is not recognized in sm
    df = df.join(comp.trend).join(comp.seasonal).join(comp.resid)
    df['resid zscore'] = (df['resid'] - df['resid'].mean()).div(df['resid'].std())
    df['trend pct_change'] = df.trend.pct_change()
    df['trend pct_change zscore'] = (df['trend pct_change'] - df['trend pct_change'].mean()).div(df['trend pct_change'].std())
    #df['trend pct_change cusum'] = df['trend pct_change'].cumsum()
    #df.dropna(inplace=True)
    #comp.plot()
    #plt.show()
    #df['trend pct_change zscore'].plot()
    #plt.show()
    #print(df.head())
    #print(comp.seasonal)
    data = data.append(df.dropna())

data

,Segment ID,Speed(miles/hour),Ref Speed(miles/hour),Travel Time(Minutes),CValue,trend,seasonal,resid,resid zscore,trend pct_change,trend pct_change zscore
Date Time,,,,,,,,,,,
2021-11-08 01:00:00,1.237051e+09,28.00,28.0,1.1300,84.000000,1.163140,-0.031615,-0.001525,-0.006481,-2.220446e-16,0.003496
2021-11-08 02:00:00,1.237051e+09,28.00,28.0,1.1300,84.000000,1.163140,-0.031615,-0.001525,-0.006481,0.000000e+00,0.003496
2021-11-08 03:00:00,1.237051e+09,28.00,28.0,1.1275,99.000000,1.163132,0.010445,-0.046077,-0.675593,-6.396889e-06,-0.010836
2021-11-08 04:00:00,1.237051e+09,30.00,28.0,1.0600,90.000000,1.163065,-0.068760,-0.034305,-0.498794,-5.757237e-05,-0.125490
2021-11-08 05:00:00,1.237051e+09,32.75,28.0,0.9650,96.670000,1.162909,-0.180032,-0.017878,-0.252079,-1.343433e-04,-0.297488
...,...,...,...,...,...,...,...,...,...,...,...
2021-12-14 19:00:00,3.859743e+08,13.00,13.0,0.2700,88.223333,0.253482,0.009339,0.007179,0.469278,5.580193e-04,1.120833
2021-12-14 20:00:00,3.859743e+08,11.50,13.0,0.3175,88.223333,0.253512,0.052255,0.011733,0.759181,1.174122e-04,0.256078
2021-12-14 21:00:00,3.859743e+08,13.00,13.0,0.2700,88.223333,0.253512,0.011422,0.005066,0.334747,-2.220446e-16,0.025640


In [8]:
data.to_parquet('//scdata2/signalshar/Data_Analysis/Data/Performance/travel_time.parquet')